# Delta Table Maintenance Part 1 - Prep Notebook
This notebook is used to query spark for the listing of Delta Tables so it can be passed onto a subsequent pipeline activity for Notebook maintenance.


#### Declare ADLS Gen2 value to pass to the DataFrame

In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 

# Primary storage info 
account_name = 'demosynapsestoragejb' # fill in your primary account name that is associated with the Synapse WOrkspace
container_name = 'other' # fill in your container name, you need to manually create this in your ADLS Gen2 account.
relative_path = 'maintenance/DeltaTablesList/' # fill in your relative folder path 

adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (container_name, account_name, relative_path) 
print('Primary storage account path: ' + adls_path) 

#### Use the snippet below to request a listing of the Delta tables to be passed to the dataframe

In [ ]:
tablesDF = spark.sql("SHOW TABLES")
tablesDF.show()

#### Filter out all temporary tables by tableName
This is an extra step to filter the listing if you have temp tables.  They may be named differently than mine and your wuery may need to be adjusted.

In [ ]:
dtlDF = tablesDF.where("NOT(tableName like 'tmp%')")
dtlDF.show()

#### Write the Datafame to the ADLS Gen 2 folder.  
The repartition command keeps everything as one file.

In [ ]:
dtlDF.repartition(1) \
    .write.csv(adls_path, mode = 'overwrite', header = 'true')
    # .write.parquet(adls_path, mode = 'overwrite') 